## config

In [1]:
# banknote classification config

# 超参配置
# yaml
class Hyperparameter:
    # ################################################################
    #                             Data
    # ################################################################
    device = 'cpu'
    data_root = './data/'
    origin_bgn_root = '../input/gru-speech-command-prepare/_background_noise_'
    bgn_root = './data/bgn'
    cls_mapper_path = './data/cls_mapper.json'

    metadata_train_path = './data/train_speech_commands.txt'
    metadata_eval_path = './data/eval_speech_commands.txt'
    metadata_test_path = './data/test_speech_commands.txt'

    cls_name_list = ['bgn', 'down', 'go', 'left', 'off', 'on', 'right', 'stop']
    cls_folder_name_list = [
        './data/bgn',
        '../input/gru-speech-command-prepare/down',
        '../input/gru-speech-command-prepare/go',
        '../input/gru-speech-command-prepare/left',
        '../input/gru-speech-command-prepare/off',
        '../input/gru-speech-command-prepare/on',
        '../input/gru-speech-command-prepare/right',
        '../input/gru-speech-command-prepare/stop'
    ]

    class_num = 8
    mel_size = 40
    seed = 1234  # random seed

    # ################################################################
    #                             Model Structure
    # ################################################################
    data_point_channel = mel_size
    rnn_hidden_dim = 256
    rnn_layer_num = 2
    is_bidirection = True
    fc_drop = 0.3

    # ################################################################
    #                             Experiment
    # ################################################################
    batch_size = 8
    init_lr = 5e-4
    epochs = 5
    verbose_step = 50
    save_step = 500


HP = Hyperparameter()


## utils

In [2]:
import os
from PIL import Image
import torchaudio

def gen_cls_mapper(cls_name_list):

    cls_mapper = {
        'cls2id': {},
        'id2cls': {}
    }

    for i, name in enumerate(cls_name_list):
        cls_mapper['cls2id'][name] = i
        cls_mapper['id2cls'][i] = name

    return cls_mapper

# 获取某个文件夹下面所有后缀为suffix的文件，返回path的list
def recursive_fetching(root, suffix=['jpg', 'png']):
    all_file_path = []

    def get_all_files(path):
        all_file_list = os.listdir(path)
        # 遍历该文件夹下的所有目录或者文件
        for file in all_file_list:
            filepath = os.path.join(path, file)
            # 如果是文件夹，递归调用函数
            if os.path.isdir(filepath):
                get_all_files(filepath)
            # 如果不是文件夹，保存文件路径及文件名
            elif os.path.isfile(filepath):
                all_file_path.append(filepath)

    get_all_files(root)

    file_paths = [it for it in all_file_path if os.path.split(it)[-1].split('.')[-1].lower() in suffix]

    return file_paths


def load_meta(meta_path):
    with open(meta_path, 'r') as fr:
        return [line.strip().split('|') for line in fr.readlines()]


def load_mel(audio_path):
    wave, sampling_rate = torchaudio.load(audio_path)
    mel = torchaudio.transforms.MelSpectrogram(sample_rate=sampling_rate, n_mels=40)(wave).squeeze(0)
    return mel


## preprocess

In [3]:
import os
import random
import json
from scipy.io import wavfile as wf

random.seed(HP.seed)

for foldername in ['data', 'log', 'model_save', 'data/bgn']:
    if not os.path.exists(foldername):
        os.mkdir(foldername)

def chop_bgn():
    '''
    将原始的噪声，以200ms为步长剪切为多个长度为1000ms的噪声片段
    '''

    wav_files = recursive_fetching(HP.origin_bgn_root, suffix=['wav', 'WAV'])

    for wav_file in wav_files:
        file_name = os.path.split(wav_file)[-1]
        sampling_rate, data = wf.read(wav_file)
        data_len = data.shape[0]
        len_200ms = int(sampling_rate/5) # sampling_rate * 200 / 1000
        count = round(data_len/len_200ms)

        for i in range(count):
            segment = data[i*len_200ms:i*len_200ms+sampling_rate]
            output_file_name = "seg-%04d-%s" % (i, file_name)
            wf.write(os.path.join(HP.bgn_root, output_file_name), sampling_rate, segment)

chop_bgn()

# 构建类别到id的映射
cls_mapper = gen_cls_mapper(HP.cls_name_list)
json.dump(cls_mapper, open(HP.cls_mapper_path, 'w'))

# 获取train和test的数据集，并将它们合并
dataset = []
for cls_folder_name in HP.cls_folder_name_list:
    dataset.extend(recursive_fetching(cls_folder_name, suffix=['wav']))
dataset_num = len(dataset)
print("Number of total items is", dataset_num)
random.shuffle(dataset)

dataset_dict = {}
for it in dataset:
    cls_name = os.path.split(os.path.split(it)[0])[-1]
    cls_id = cls_mapper['cls2id'][cls_name]
    if cls_id not in dataset_dict:
        dataset_dict[cls_id] = [it]
    else:
        dataset_dict[cls_id].append(it)

# 自己划分训练集、评价集和测试集
train_ratio, eval_ratio, test_ratio = 0.8, 0.1, 0.1
train_set, eval_set, test_set = [], [], [],
for _, set_list in dataset_dict.items():
    length = len(set_list)
    train_num, eval_num = int(length * train_ratio), int(length * eval_ratio)
    test_num = length - train_num - eval_num
    random.shuffle(set_list)
    train_set.extend(set_list[:train_num])
    eval_set.extend(set_list[train_num:train_num + eval_num])
    test_set.extend(set_list[train_num + eval_num:])

# 再次随机打乱
random.shuffle(train_set)
random.shuffle(eval_set)
random.shuffle(test_set)

print('num of trainset : %d' % (len(train_set)))
print('num of evalset : %d' % (len(eval_set)))
print('num of testset : %d' % (len(test_set)))

with open(HP.metadata_train_path, 'w') as fw:
    for path in train_set:
        fn_start = os.path.split(os.path.split(path)[0])[-1]
        cls_id = cls_mapper['cls2id'][fn_start]
        fw.write('%d|%s\n' % (cls_id, path))

with open(HP.metadata_eval_path, 'w') as fw:
    for path in eval_set:
        fn_start = os.path.split(os.path.split(path)[0])[-1]
        cls_id = cls_mapper['cls2id'][fn_start]
        fw.write('%d|%s\n' % (cls_id, path))

with open(HP.metadata_test_path, 'w') as fw:
    for path in test_set:
        fn_start = os.path.split(os.path.split(path)[0])[-1]
        cls_id = cls_mapper['cls2id'][fn_start]
        fw.write('%d|%s\n' % (cls_id, path))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:21: WavFileWarning: Chunk (non-data) not understood, skipping it.


Number of total items is 28835
num of trainset : 23065
num of evalset : 2880
num of testset : 2890


## dataset_kws

In [4]:
import torch
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

class KWSDataset(torch.utils.data.Dataset):

    def __init__(self, metadata_path):
        self.dataset = load_meta(metadata_path)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        cls_id, path = int(item[0]), item[1]
        mel = load_mel(path) # [data_point_dim, sequence_len] = [40, ?]
        # [x,x,x,x,x,0,0]
        # [x,x,x,x,x,x,x]
        return mel.to(HP.device), cls_id # cls_int

    def __len__(self):
        return len(self.dataset)


# batch : 8
def collate_fn(batch):
    # [(mel cls_id),(mel cls_id),(mel cls_id),(mel cls_id)...]
    sorted_batch = sorted(batch, key=lambda b: b[0].size(1), reverse=True)
    # get all mel and pad them: mel defaul dim: [40, ?]=[datapoint_dim, L] -> [L, datapoint_dim]
    mel_list = [item[0].transpose(0, 1) for item in sorted_batch]
    # [sequence, batch, datapoint_dim], [batch, sequence, datapoint_dim]
    mel_padded = pad_sequence(mel_list, batch_first=True)
    labels = torch.LongTensor([item[1] for item in sorted_batch]) # transfer labels to long tensor
    mel_lengths = torch.LongTensor([item.size(0) for item in mel_list])
    return mel_padded, mel_lengths, labels


## model

In [5]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence


class SpeechCommandModel(nn.Module):

    def __init__(self):
        super(SpeechCommandModel, self).__init__()

        self.rnn = nn.GRU(
            input_size=HP.data_point_channel,
            hidden_size=HP.rnn_hidden_dim,
            num_layers=HP.rnn_layer_num,
            bidirectional=HP.is_bidirection
        )

        fc_in_dim = 2 * HP.rnn_hidden_dim if HP.is_bidirection else HP.rnn_hidden_dim

        self.fc = nn.Sequential(
            nn.Linear(fc_in_dim, 1024),
            nn.Mish(),
            nn.Dropout(HP.fc_drop),
            nn.Linear(1024, 512),
            nn.Mish(),
            nn.Dropout(HP.fc_drop),
            nn.Linear(512, HP.class_num)
        )

    def forward(self, mel_input, mel_lengths):
        mel_input = mel_input.permute(1, 0, 2)
        mel_packed = pack_padded_sequence(mel_input, mel_lengths)
        output_packed, hn = self.rnn(mel_packed)
        output, _ = pad_packed_sequence(output_packed)

        # if语句在模型迁移后使用可能出现bug，此处这么写仅为学习，实际模型搭建过程中应避免
        if HP.is_bidirection:
            forward_feature = output[-1, :, :HP.rnn_hidden_dim]
            backward_feature = output[0, :, HP.rnn_hidden_dim:]
            fc_in = torch.cat((forward_feature, backward_feature), dim=-1)
            cls_output = self.fc(fc_in)
        else:
            cls_output = self.fc(output[-1])
        
        return cls_output


## inference

In [6]:
import torch
from torch.utils.data import DataLoader

model = SpeechCommandModel()
checkpoint = torch.load('../input/gru-speech-commands-model/model_3_9500.model', map_location='cpu')
model.load_state_dict(checkpoint['model_state_dict'])

testset = KWSDataset(HP.metadata_test_path)
test_loader = DataLoader(testset, batch_size=HP.batch_size, shuffle=True, drop_last=False, collate_fn=collate_fn)

model.eval()

total_count = 0
correct_count = 0

with torch.no_grad():
    for batch in test_loader:
        x, x_lens, y = batch
        pred = model(x, x_lens)
        print("Actual:", y.data.cpu().numpy().tolist())  # actual class label
        print("  Pred:", torch.argmax(pred, dim=-1).data.cpu().numpy().tolist())  # prediction label
        total_count += pred.size(0)
        correct_count += (torch.argmax(pred, 1) == y).sum()

print('Acc: %.3f' % (correct_count / total_count))

Actual: [4, 0, 1, 6, 6, 5, 1, 1]
  Pred: [4, 0, 1, 6, 6, 5, 2, 1]
Actual: [5, 4, 1, 2, 7, 3, 1, 0]
  Pred: [5, 3, 1, 2, 7, 3, 1, 0]
Actual: [4, 3, 5, 4, 2, 2, 6, 7]
  Pred: [4, 3, 5, 4, 2, 2, 6, 7]
Actual: [1, 7, 0, 0, 7, 5, 4, 7]
  Pred: [1, 7, 0, 0, 7, 5, 4, 7]
Actual: [7, 7, 1, 6, 2, 3, 6, 1]
  Pred: [7, 7, 1, 6, 2, 3, 6, 1]
Actual: [6, 3, 4, 7, 1, 1, 1, 2]
  Pred: [6, 3, 4, 1, 1, 1, 1, 2]
Actual: [3, 4, 5, 0, 3, 4, 6, 5]
  Pred: [3, 4, 5, 0, 3, 4, 7, 5]
Actual: [2, 2, 7, 2, 3, 2, 3, 4]
  Pred: [2, 2, 7, 2, 3, 2, 3, 4]
Actual: [6, 4, 2, 3, 2, 2, 6, 4]
  Pred: [3, 4, 2, 3, 2, 2, 6, 4]
Actual: [7, 3, 2, 5, 3, 7, 1, 3]
  Pred: [7, 3, 2, 5, 3, 3, 2, 3]
Actual: [1, 4, 4, 5, 4, 3, 4, 2]
  Pred: [1, 4, 4, 5, 4, 3, 4, 2]
Actual: [4, 4, 2, 6, 1, 7, 6, 2]
  Pred: [4, 4, 2, 6, 1, 7, 6, 2]
Actual: [4, 5, 4, 7, 1, 7, 5, 5]
  Pred: [4, 5, 4, 7, 1, 7, 2, 5]
Actual: [3, 7, 5, 3, 5, 6, 1, 4]
  Pred: [3, 7, 4, 3, 5, 5, 2, 4]
Actual: [2, 1, 3, 5, 3, 4, 2, 6]
  Pred: [2, 1, 3, 5, 3, 4, 2, 6]
Actual: [1